# Setup

In [ ]:
# PLEASE READ:
# This notebook here is mostly compiling the results AFTER the benchmarks are
# done. If you use this notebook, please install LISA
#     https://github.com/ARM-software/lisa
# and then workload-processor
#     https://github.com/mrkajetanp/workload-processor.git
# first. After the installation, run this workload via:
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_speedometer.yaml baseline
# $ workload-processor run $WORKLOAD_PROCESSOR_INSTALL_DIR/agendas/agenda_speedometer.yaml test
# This will give you two directories of
#     speedometer_baseline_$RUNS_$DATE
# and
#     speedometer_test_$RUNS_$DATE
# To obtain power consumption results on the two benchmark runs, do
# $ workload-processor process speedometer_baseline_$RUNS_$DATE -m power
# and then
# $ workload-processor process speedometer_test_$RUNS_$DATE -m power
#
# Only after this should you start using this notebook. The reason is that
# these benchmark runs generate a huge amount of traces. Running the benchmarks
# inside this notebook can easily crash Jupyter completely.

In [ ]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

## Runs

In [24]:
sp = WorkloadNotebookAnalysis('/data_sda/honxia02/lisa/', [
    'speedometer_mainoffice_10_0711',
    'speedometer_sumofficediv4_10_0711',
    'speedometer_sumofficediv3.2_10_0711',
])

sp.show()

,metric,value,unit,iteration,workload,id,wa_path,kernel,tag
0,Speedometer Score,79.09,Runs per minute,1,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
1,Speedometer Score,79.41,Runs per minute,2,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
2,Speedometer Score,78.74,Runs per minute,3,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
3,Speedometer Score,79.69,Runs per minute,4,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
4,Speedometer Score,79.28,Runs per minute,5,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
5,Speedometer Score,79.57,Runs per minute,6,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
6,Speedometer Score,79.73,Runs per minute,7,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
7,Speedometer Score,78.70,Runs per minute,8,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
8,Speedometer Score,79.14,Runs per minute,9,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice
9,Speedometer Score,79.70,Runs per minute,10,speedometer,wk1,speedometer_mainoffice_10_0711,5.18.0-armpower-00029-gc35255a0e739,mainoffice


benchmark_dirs: ['speedometer_mainoffice_10_0711', 'speedometer_sumofficediv4_10_0711', 'speedometer_sumofficediv3.2_10_0711']
tags: ['mainoffice', 'sumofficediv4', 'sumofficediv3.2']
kernels: ['5.18.0-armpower-00029-gc35255a0e739', '5.18.0-armpower-00036-g7f9273f79788', '5.18.0-armpower-00036-g769ac0cc40ef']


# Benchmark scores

## Line plot

In [ ]:
sp.plot.results_line(['Speedometer Score'])

## Bar plot

In [25]:
sp.plot.results_bar(['Speedometer Score'], width=1000)

+-------------------+-----------------+-------+-----------+
|      metric       |       tag       | value | perc_diff |
+-------------------+-----------------+-------+-----------+
| Speedometer Score |   mainoffice    | 79.3  |   0.0%    |
| Speedometer Score |  sumofficediv4  | 79.8  |   0.61%   |
| Speedometer Score | sumofficediv3.2 | 79.7  |   0.53%   |
+-------------------+-----------------+-------+-----------+


# Overutilized

## Line plot

In [ ]:
sp.plot.overutilized_line()

# Perf

## Line plot

In [ ]:
sp.plot.perf_line()

## Bar plot

In [ ]:
sp.plot.perf_bar()

# Idle residency

## Bar plot

In [ ]:
sp.plot.idle_residency_bar()

# Idle misses

## Bar plot

In [ ]:
sp.plot.idle_miss_bar()

# Power usage

## Line plot

In [ ]:
sp.plot.power_meter_line()

## Bar plot

In [26]:
sp.plot.power_meter_bar()

+------------+--------+-----------------+--------+-----------+
|  channel   | metric |       tag       | value  | perc_diff |
+------------+--------+-----------------+--------+-----------+
|    CPU     | gmean  |   mainoffice    | 1858.7 |   0.0%    |
|  CPU-Big   | gmean  |   mainoffice    | 1455.9 |   0.0%    |
| CPU-Little | gmean  |   mainoffice    | 353.0  |   0.0%    |
|  CPU-Mid   | gmean  |   mainoffice    |  49.8  |   0.0%    |
|    GPU     | gmean  |   mainoffice    |  14.5  |   0.0%    |
|   Total    | gmean  |   mainoffice    | 1873.2 |   0.0%    |
|    CPU     | gmean  |  sumofficediv4  | 1848.0 |  -0.58%   |
|  CPU-Big   | gmean  |  sumofficediv4  | 1429.1 |  -1.84%   |
| CPU-Little | gmean  |  sumofficediv4  | 363.4  |   2.93%   |
|  CPU-Mid   | gmean  |  sumofficediv4  |  55.2  |  10.83%   |
|    GPU     | gmean  |  sumofficediv4  |  14.6  |   1.26%   |
|   Total    | gmean  |  sumofficediv4  | 1862.6 |  -0.57%   |
|    CPU     | gmean  | sumofficediv3.2 | 1903.2 |   2.

# Energy estimate

## Line plot

In [ ]:
sp.plot.energy_estimate_line()

## Bar plot

In [ ]:
sp.plot.energy_estimate_bar()

# Thermal

## Line plot

In [ ]:
sp.plot.thermal_line()

## Bar plot

In [ ]:
sp.plot.thermal_bar()

# Frequency

## Line plot

In [ ]:
sp.plot.frequency_line()

## Bar plot

In [ ]:
sp.plot.frequency_bar()

# CFS signals

## Line plot

In [ ]:
sp.plot.sched_pelt_cfs_line()

## Bar plot

In [ ]:
sp.plot.sched_pelt_cfs_bar()

# Wakeup latency - tasks

## Line plot

In [ ]:
sp.plot.wakeup_latency_line(height=1100, columns=4)

## Bar plot

In [ ]:
sp.plot.wakeup_latency_bar(height=1000, columns=4)

## Quantiles

In [ ]:
sp.plot.wakeup_latency_quantiles_bar(height=2000, columns=2)

# Wakeup latency - cgroups

## Line plot

In [ ]:
sp.plot.wakeup_latency_cgroup_line()

## Bar plot

In [ ]:
sp.plot.wakeup_latency_cgroup_bar()

## Quantiles

In [ ]:
sp.plot.wakeup_latency_cgroup_quantiles_bar()

# CPU residency - tasks

## Clusters - Line plot

In [ ]:
sp.plot.tasks_cpu_residency_cluster_line()

## Clusters - Bar plot

In [ ]:
sp.plot.tasks_cpu_residency_cluster_bar()

## Clusters - Per-task bar plot

In [ ]:
sp.plot.tasks_cpu_residency_per_task_bar()

## CPUs - Bar plot

In [ ]:
sp.plot.tasks_cpu_residency_cpu_bar()

# CPU residency - cgroups

## Clusters - Bar plot

In [ ]:
sp.plot.cgroup_cpu_residency_cluster_bar()

## CPUs - Bar plot

In [ ]:
sp.plot.cgroup_cpu_residency_cpu()

# Summary - TLDR

In [ ]:
sp.plot.summary()

## Save image plots

In [ ]:
sp.save_image_plots('/home/kajpuc01/temp/sp_vh_impact')

In [23]:
main_avg_time = 83.32359989999998
sum_avg_time = 83.07767630000001
0.9942 * sum_avg_time / main_avg_time

0.9912656903516722